In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 79kB/s 
     |████████████████████████████████| 204kB 16.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=61749e228f8841e9899ba66b198dc5a903733b5c2ac1679c5cb4d44537904b39
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)

In [ ]:
from pyspark.sql import SparkSession as ss
spark = ss.builder.getOrCreate()

In [ ]:
df = spark.read.option("header","true").csv('/content/drive/My Drive/Combined Annotated/dataset_final_with_uid',multiLine=True,inferSchema = True)

In [ ]:
df.printSchema()

root
 |-- u_id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- pre_text_vader: string (nullable = true)
 |-- pre_text_all_upd: string (nullable = true)
 |-- mean_sent_score: double (nullable = true)
 |-- final_sent_class: string (nullable = true)



In [ ]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors,VectorUDT
class generate_embeddings:
  def __init__(self,df):
      self.df = df.select("u_id","text","pre_text_vader","pre_text_all_upd","mean_sent_score","final_sent_class")
  
  @staticmethod
  @udf(returnType = VectorUDT())
  def embeddings(text):
    res = model([text]).numpy().tolist()
    if len(res) == 1:
      res = res.pop()
    

  def process(self):
    # self.df = self.df.withColumn("sb_text",generate_embeddings.embeddings("text"))
    # self.df = self.df.withColumn("sb_pre_text_vader",generate_embeddings.embeddings("pre_text_vader"))
    self.df = self.df.withColumn("USE_pre_text_all_upd",generate_embeddings.embeddings("pre_text_all_upd"))

    # self.df1 = self.df1.withColumn("sb_text",generate_embeddings.embeddings("text"))
    # self.df1 = self.df1.withColumn("sb_pre_text_vader",generate_embeddings.embeddings("pre_text_vader"))
    # self.df1 = self.df1.withColumn("sb_pre_text_all_upd",generate_embeddings.embeddings("pre_text_all_upd"))

    self.df.write.option("header","true").csv('/content/drive/My Drive/Annotated/Combined Annotated/USE_covid_dataset_with_uid')
    # self.df1.write.mode("overwrite").option("header","true").csv('/content/drive/My Drive/Annotated/mean_sent_lex/final_sent_class/Combined Annotated/sb_covid_dataset_final_with_uid')


In [ ]:
type(model(["Hi"]).numpy().tolist().pop())

list

In [ ]:
g = generate_embeddings(df)
g.process()

PickleError: ignored